The points distribution for this case is as follows:
1. Read the dataset
2. Drop the columns which are unique for all users like IDs (2.5 points)
3. Distinguish the feature and target set (2.5 points)
4. Divide the data set into Train and test sets
5. Normalize the train and test data (2.5 points)
6. Initialize &amp; build the model (10 points)
7. Optimize the model (5 points)
9. Predict the results using 0.5 as a threshold (5 points)
10. Print the Accuracy score and confusion matrix (2.5 points)

In [1]:
import tensorflow as tf
import keras

tf.reset_default_graph()
tf.set_random_seed(42)

Using TensorFlow backend.


In [2]:
# Importing Helper Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

In [3]:
# Reading Dataset
df = pd.read_csv('bank.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.shape

(10000, 14)

In [5]:
print('Number of unique customer ids: ',len(df.CustomerId.unique()))
print('Number of unique Surnames: ',len(df.Surname.unique()))
print('Number of unique Geography: ',len(df.Geography.unique()))

Number of unique customer ids:  10000
Number of unique Surnames:  2932
Number of unique Geography:  3


In [6]:
# We can remove Customer Id and Row number, as they are completely unique
# We can remover Surnames too, as name does not have any impact on the model
df_new = df.drop(['CustomerId','RowNumber', 'Surname'], axis=1)
df_new.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
# We need to encode certain features - Geography, Gender
from sklearn.preprocessing import LabelBinarizer
geo_lb = LabelBinarizer()
G = geo_lb.fit_transform(df_new.Geography.values)
df_new['France'] = G[:,0]
df_new['Germany'] = G[:,1]
df_new['Spain'] = G[:,2]
gender_lb = LabelBinarizer()
df_new['GenderCode'] = gender_lb.fit_transform(df_new.Gender.values)
df_new.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,GenderCode
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,1,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1,0


In [9]:
df_new.drop(['Gender','Geography'], axis=1, inplace=True)
df_new.head(5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,GenderCode
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,0


In [10]:
X_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 
              'EstimatedSalary', 'France', 'Germany', 'Spain', 'GenderCode']

Y_Label = ['Exited']

X = df_new[X_features]
Y = df_new[Y_Label]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=None)

In [13]:
print('Train Data shape:', X_train.shape)
print('Test Data shape:', X_test.shape)

Train Data shape: (8000, 12)
Test Data shape: (2000, 12)


In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#Standardizing Test and Train data
X_train =  sc.fit_transform(X_train)
X_test =  sc.fit_transform(X_test)

In [15]:
#Y_test = tf.keras.utils.to_categorical(Y_test)
#Y_train = tf.keras.utils.to_categorical(Y_train)

In [16]:
# Building Model 1
model = keras.Sequential([
    keras.layers.Dense(100,activation='relu', input_shape=(12,)),
    keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.fit(X_train, Y_train, epochs=5, batch_size=128)

Epoch 1/5
8000/8000 [==============================] - 0s 22us/step - loss: 0.6295 - acc: 0.6933
Epoch 2/5
8000/8000 [==============================] - 0s 9us/step - loss: 0.5260 - acc: 0.7945
Epoch 3/5
8000/8000 [==============================] - 0s 9us/step - loss: 0.4900 - acc: 0.7953
Epoch 4/5
8000/8000 [==============================] - 0s 8us/step - loss: 0.4689 - acc: 0.7976
Epoch 5/5
8000/8000 [==============================] - 0s 9us/step - loss: 0.4551 - acc: 0.8049


In [18]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('test_acc:', test_acc)

2000/2000 [==============================] - 0s 29us/step
test_acc: 0.8145


In [19]:
# Building Model 2
# With Batch Normalization

model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Dense(100,activation='relu', input_shape=(12,)))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dense(2, activation='softmax'))

model2.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model2.fit(X_train, Y_train, epochs=5, batch_size=128)

Epoch 1/5
8000/8000 [==============================] - 0s 39us/step - loss: 0.6864 - acc: 0.6545
Epoch 2/5
8000/8000 [==============================] - 0s 13us/step - loss: 0.4974 - acc: 0.7752
Epoch 3/5
8000/8000 [==============================] - 0s 13us/step - loss: 0.4448 - acc: 0.8085
Epoch 4/5
8000/8000 [==============================] - 0s 12us/step - loss: 0.4197 - acc: 0.8221
Epoch 5/5
8000/8000 [==============================] - 0s 13us/step - loss: 0.4051 - acc: 0.8304


In [21]:
test_loss, test_acc = model2.evaluate(X_test, Y_test)
print('test_acc:', test_acc)

2000/2000 [==============================] - 0s 53us/step
test_acc: 0.8325


In [27]:
# Improving Model 2's performance with SDG optimizer

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.001)

model2.compile(optimizer=sgd_optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model2.fit(X_train, Y_train, epochs=5, batch_size=128)

Epoch 1/5
8000/8000 [==============================] - 0s 14us/step - loss: 0.3137 - acc: 0.8696
Epoch 2/5
8000/8000 [==============================] - 0s 12us/step - loss: 0.3162 - acc: 0.8694
Epoch 3/5
8000/8000 [==============================] - 0s 12us/step - loss: 0.3121 - acc: 0.8685
Epoch 4/5
8000/8000 [==============================] - 0s 12us/step - loss: 0.3132 - acc: 0.8688
Epoch 5/5
8000/8000 [==============================] - 0s 11us/step - loss: 0.3133 - acc: 0.8679


In [36]:
test_loss, test_acc = model2.evaluate(X_test, Y_test)
print('test_acc:', test_acc)

2000/2000 [==============================] - 0s 16us/step
test_acc: 0.875


In [38]:
Y_pred = model2.predict(X_test)

In [59]:
Y_pred[:,0]

array([3.5358143e-01, 9.7836429e-01, 6.1039770e-01, ..., 4.7538491e-05,
       9.6482021e-01, 8.7264651e-01], dtype=float32)

In [66]:
Y_pred_df = pd.DataFrame(Y_pred[:,0])

In [67]:
Y_pred_df.shape

(2000, 1)

In [68]:
Y_pred_df.head(5)

,0
0,0.353581
1,0.978364
2,0.610398
3,0.667484
4,0.900839


In [87]:
Y_pred_df['Exited'] = Y_pred_df[0]

In [114]:
for i in range(0,len(Y_pred_df),1):
    result = Y_pred_df[Y_pred_df.index == i][0] > 0.5
    if(result.item() == True):
        Y_pred_df.loc[i, 'Exited'] = 0
    else:
        Y_pred_df.loc[i, 'Exited'] = 1

In [115]:
Y_pred_df['Exited'].head(5)

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Exited, dtype: float32

In [116]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, Y_pred_df['Exited'])

array([[1543,   63],
       [ 187,  207]], dtype=int64)

In [121]:
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, Y_pred_df['Exited'],normalize=False))
print(accuracy_score(Y_test, Y_pred_df['Exited'],normalize=True))

1750
0.875


### Summary:
1. Number of Items the model predicted correctly are 1750 out of 2000
2. Accuracy Score is 87.5%
3. That's similar to the accuracy of the tensor model  